In [2]:
import pandas as pd
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
'''fem = pd.read_excel('/content/drive/My Drive/atuaria/tabua.xlsx', header=3, sheet_name=0)
masc = pd.read_excel('/content/drive/My Drive/atuaria/tabua.xlsx', header=3, sheet_name=1)
ambos = pd.read_excel('/content/drive/My Drive/atuaria/tabua.xlsx', header=3, sheet_name=2)

tabuas=[fem,masc,ambos]'''

"fem = pd.read_excel('/content/drive/My Drive/atuaria/tabua.xlsx', header=3, sheet_name=0)\nmasc = pd.read_excel('/content/drive/My Drive/atuaria/tabua.xlsx', header=3, sheet_name=1)\nambos = pd.read_excel('/content/drive/My Drive/atuaria/tabua.xlsx', header=3, sheet_name=2)\n\ntabuas=[fem,masc,ambos]"

In [4]:
at2000 = pd.read_excel('/content/drive/My Drive/atuaria/tabua2.xlsx', header=0)
tabuas=[at2000]

In [5]:
i=0.06
for tabua in tabuas:
#    tabua = tabua.rename({'Idades Exatas\n(x)': 'x'}, axis=1)
    tabua['px']=1-tabua['qx']
    tabua['dx']=tabua['qx']*tabua['lx']
    tabua['Dx']=tabua['lx']*((1/(1+i))**tabua.index)
    tabua['Cx']=tabua['dx']*((1/(1+i))**(tabua.index + 1))
    Nx=[]
    Sx=[]
    Mx=[]
    Rx=[]

    for n in tabua.index:
        Nx.append(tabua['Dx'].iloc[n:].sum())
        Mx.append(tabua['Cx'].iloc[n:].sum())

    tabua['Nx']=Nx
    tabua['Mx']=Mx

    for n in tabua.index:
        Sx.append(tabua['Nx'].iloc[n:].sum())
        Rx.append(tabua['Mx'].iloc[n:].sum())

    tabua['Sx']=Sx
    tabua['Rx']=Rx


In [6]:
tabua = tabuas[0]

In [7]:
def anuidade(x,dif,temp,antecipada,benef_frac,premio,crescente):
    ''' 
    x = idade
    dif = anos de diferimento
    temp = se for temporária, informar a quantidade de anos, se vitalícia temp=0
    antecipada = 's' para sim, 'n' para não, para informar se é uma anuidade antecipada
    benef_frac = benefício fracionado, em t pagamentos, se não fracionado t = 1
    premio = valor do prêmio
    crescente = pagamentos crescentes em progressão aritmética, 's' para sim, 'n' para não
    '''
    convert = 1

    if antecipada == 'n':
        convert = -1
        dif=dif+1

    if x+dif+temp > tabua['x'].iloc[-1]:
        raise Exception(f"Horizonte de idades para apuração excede a idade ômega, {tabua['x'].iloc[-1]} anos")

    if crescente == 'n':
        a = tabua['Nx']
        b = 0
        c = 0 
        d = tabua['Dx']
    elif crescente == 's':
        a = tabua['Sx']
        b = temp * tabua['Nx'][x+dif+temp]
        c = temp * tabua['Dx'][x+dif+temp]
        d = tabua['Nx']
    else:
        print("Informe 's' ou 'n' para crescente, quinta variável da função")       
    
    if temp > 0:
        numerador = a[x+dif]-a[x+dif+temp]
        dx=d[x+dif]-d[x+dif+temp]
    elif temp == 0:
        numerador = a[x+dif]
        dx=d[x+dif]
    
    fracao =  (((benef_frac-1)/(2*benef_frac))*((dx-c)/tabua['Dx'][x]))
    fracao = fracao * convert
    
    beneficio = ((numerador-b)/tabua['Dx'][x]) - fracao
    anuidade = premio * beneficio

    return anuidade

def anuidade_p_periodico(x,dif,temp,antecipada,benef_frac,premio,crescente,p_antecipado,p_temp,p_fracionado):
    ''' 
    x = idade
    dif = anos de diferimento
    antecipada = 's' para sim, 'n' para não, para informar se é uma anuidade antecipada
    temp = se for temporária, informar a quantidade de anos, se vitalícia temp=0
    benef_frac = benefício fracionado, em t meses
    premio = valor do prêmio
    crescente = pagamentos crescentes em progressão aritmética, 's' para sim, 'n' para não
    p_antecipado = 's' ou 'n' para informar se os prêmios serão ou não antecipados
    p_temp = informar a quantidade de anos em que os prêmios serão pagos
    p_fracionado = informar o fracionamento anual dos prêmios (1=anual, 6 = bimestral, 12 = mensal, 2 = semestral)
    '''
    convert = 1

    if antecipada == 'n':
        convert = -1
        dif=dif+1

    if x+dif+temp > tabua['x'].iloc[-1]:
        raise Exception(f"Horizonte de idades para apuração excede a idade ômega, {tabua['x'].iloc[-1]} anos")

    if crescente == 'n':
        a = tabua['Nx']
        b = 0
        c = 0 
        d = tabua['Dx']
    elif crescente == 's':
        a = tabua['Sx']
        b = temp * tabua['Nx'][x+dif+temp]
        c = temp * tabua['Dx'][x+dif+temp]
        d = tabua['Nx']
    else:
        print("Informe s ou n para crescente, quinta variável da função")       
    
    if temp > 0:
        numerador = a[x+dif]-a[x+dif+temp]
        dx=d[x+dif]-d[x+dif+temp]
    elif temp == 0:
        numerador = a[x+dif]
        dx=d[x+dif]

    benef_fracao = ((benef_frac-1)/(2*benef_frac))*(dx/tabua['Dx'][x])
    benef_fracao * convert
    
    beneficio = (numerador / tabua['Dx'][x] ) - benef_fracao 

    if p_antecipado == 'n':
      x_p = x+1
      dx_temp = -((p_fracionado-1)/(2*p_fracionado))*(tabua['Dx'][x]-tabua['Dx'][x+p_temp])/tabua['Dx'][x]

    elif p_antecipado == 's':
      x_p = x
      dx_temp = ((p_fracionado-1)/(2*p_fracionado))*(tabua['Dx'][x]-tabua['Dx'][x+p_temp])/tabua['Dx'][x]    
    
    fracao = (tabua['Nx'][x_p]-tabua['Nx'][x_p+p_temp])/tabua['Dx'][x]

    anuidade_p_periodico = premio * (beneficio / (fracao - dx_temp))
  
    return anuidade_p_periodico


def seguro(x,dif,temp,premio,crescente):
    ''' 
    x = idade
    dif = anos de diferimento
    temp = se for temporário, informar a quantidade de anos, se vitalício temp=0
    premio = valor do prêmio
    '''
    if crescente == 'n':
        a = tabua['Mx']
    elif crescente == 's':
        a = tabua['Rx']
    else:
        print("Iforme s ou n para crescente, quinta variável da função")       

    if temp > 0:
        numerador = a[x+dif]-a[x+dif+temp]
    elif temp == 0:
        numerador = a[x+dif]

    seguro = premio * numerador/tabua['Dx'][x]
    
    return seguro

In [8]:
x=16
diferimento = 5
temporario = 10
antecipada = 's'
benef_fracionado = 12 #meses
valor_premio = 1000
crescente = 's'

anuidade(x,diferimento,temporario,antecipada,benef_fracionado,valor_premio,crescente)   #anuidade(x, diferimento, temporaria, antecipada, beneficio fracionado, valor premio, anuidade crescente aritmética)

28331.70421786608

In [9]:
anuidade_p_periodico(16,10,10,"s",1,1000,'n','n',10,12)

572.4170995520883

In [10]:
seguro(16,0,0,1000,'n')

36.507387782782274

In [11]:
df = multi_decremt = pd.read_excel('/content/drive/My Drive/atuaria/anexo.xlsx', header=0, sheet_name=1)
df['l(T)x']=1000000.00
df['d(m)x']=df['l(T)x']* (df['q`(m)x']-(df['q`(m)x']*df['q`(t)x'])/2 - (df['q`(m)x']*df['q`(d)x'])/2 - (df['q`(m)x']*df['q`(r)x'])/2 + (df['q`(m)x']*df['q`(t)x']*df['q`(d)x'])/3 + (df['q`(m)x']*df['q`(t)x']*df['q`(r)x'])/3 + (df['q`(m)x']*df['q`(d)x']*df['q`(r)x'])/3 - (df['q`(m)x']*df['q`(t)x']*df['q`(d)x']*df['q`(r)x'])/4)
df['d(t)x']=df['l(T)x']* (df['q`(t)x']-(df['q`(t)x']*df['q`(m)x'])/2 - (df['q`(t)x']*df['q`(d)x'])/2 - (df['q`(t)x']*df['q`(r)x'])/2 + (df['q`(t)x']*df['q`(m)x']*df['q`(d)x'])/3 + (df['q`(t)x']*df['q`(m)x']*df['q`(r)x'])/3 + (df['q`(t)x']*df['q`(d)x']*df['q`(r)x'])/3 - (df['q`(t)x']*df['q`(m)x']*df['q`(d)x']*df['q`(r)x'])/4)
df['d(d)x']=df['l(T)x']* (df['q`(d)x']-(df['q`(d)x']*df['q`(m)x'])/2 - (df['q`(d)x']*df['q`(t)x'])/2 - (df['q`(d)x']*df['q`(r)x'])/2 + (df['q`(d)x']*df['q`(m)x']*df['q`(t)x'])/3 + (df['q`(d)x']*df['q`(m)x']*df['q`(r)x'])/3 + (df['q`(d)x']*df['q`(t)x']*df['q`(r)x'])/3 - (df['q`(d)x']*df['q`(m)x']*df['q`(t)x']*df['q`(r)x'])/4)
df['d(r)x']=df['l(T)x']* (df['q`(r)x']-(df['q`(r)x']*df['q`(m)x'])/2 - (df['q`(r)x']*df['q`(t)x'])/2 - (df['q`(r)x']*df['q`(d)x'])/2 + (df['q`(r)x']*df['q`(m)x']*df['q`(t)x'])/3 + (df['q`(r)x']*df['q`(m)x']*df['q`(d)x'])/3 + (df['q`(r)x']*df['q`(t)x']*df['q`(d)x'])/3 - (df['q`(r)x']*df['q`(m)x']*df['q`(t)x']*df['q`(d)x'])/4)
df['d(T)x']=(df['d(r)x']+df['d(m)x']+df['d(t)x']+df['d(d)x'])

n = len(df)-1

for i in range(n):
    df['l(T)x'][i+1]=df['l(T)x'][i]-df['d(T)x'][i]
    df['d(m)x']=df['l(T)x']* (df['q`(m)x']-(df['q`(m)x']*df['q`(t)x'])/2 - (df['q`(m)x']*df['q`(d)x'])/2 - (df['q`(m)x']*df['q`(r)x'])/2 + (df['q`(m)x']*df['q`(t)x']*df['q`(d)x'])/3 + (df['q`(m)x']*df['q`(t)x']*df['q`(r)x'])/3 + (df['q`(m)x']*df['q`(d)x']*df['q`(r)x'])/3 - (df['q`(m)x']*df['q`(t)x']*df['q`(d)x']*df['q`(r)x'])/4)
    df['d(t)x']=df['l(T)x']* (df['q`(t)x']-(df['q`(t)x']*df['q`(m)x'])/2 - (df['q`(t)x']*df['q`(d)x'])/2 - (df['q`(t)x']*df['q`(r)x'])/2 + (df['q`(t)x']*df['q`(m)x']*df['q`(d)x'])/3 + (df['q`(t)x']*df['q`(m)x']*df['q`(r)x'])/3 + (df['q`(t)x']*df['q`(d)x']*df['q`(r)x'])/3 - (df['q`(t)x']*df['q`(m)x']*df['q`(d)x']*df['q`(r)x'])/4)
    df['d(d)x']=df['l(T)x']* (df['q`(d)x']-(df['q`(d)x']*df['q`(m)x'])/2 - (df['q`(d)x']*df['q`(t)x'])/2 - (df['q`(d)x']*df['q`(r)x'])/2 + (df['q`(d)x']*df['q`(m)x']*df['q`(t)x'])/3 + (df['q`(d)x']*df['q`(m)x']*df['q`(r)x'])/3 + (df['q`(d)x']*df['q`(t)x']*df['q`(r)x'])/3 - (df['q`(d)x']*df['q`(m)x']*df['q`(t)x']*df['q`(r)x'])/4)
    df['d(r)x']=df['l(T)x']* (df['q`(r)x']-(df['q`(r)x']*df['q`(m)x'])/2 - (df['q`(r)x']*df['q`(t)x'])/2 - (df['q`(r)x']*df['q`(d)x'])/2 + (df['q`(r)x']*df['q`(m)x']*df['q`(t)x'])/3 + (df['q`(r)x']*df['q`(m)x']*df['q`(d)x'])/3 + (df['q`(r)x']*df['q`(t)x']*df['q`(d)x'])/3 - (df['q`(r)x']*df['q`(m)x']*df['q`(t)x']*df['q`(d)x'])/4)
    df['d(T)x']=(df['d(r)x']+df['d(m)x']+df['d(t)x']+df['d(d)x'])

df


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


,x,q`(m)x,q`(t)x,q`(d)x,q`(r)x,l(T)x,d(m)x,d(t)x,d(d)x,d(r)x,d(T)x
0,20,0.000503,0.2431,0.0003,0,1000000.000000,441.797128,243002.407578,263.471778,0.000000,243707.676484
1,21,0.000522,0.2245,0.0003,0,756292.323516,350.419668,169717.852778,201.369198,0.000000,170269.641644
2,22,0.000544,0.2071,0.0003,0,586022.681873,285.743761,121314.087863,157.560793,0.000000,121757.392416
3,23,0.000566,0.1908,0.0003,0,464265.289456,237.671097,88543.466315,125.957912,0.000000,88907.095324
4,24,0.000591,0.1757,0.0003,0,375358.194132,202.318961,65921.057688,102.685515,0.000000,66226.062165
5,25,0.000619,0.1616,0.0003,0,309132.131968,175.865874,49932.800950,85.220666,0.000000,50193.887490
6,26,0.000650,0.1486,0.0003,0,258938.244478,155.781691,38459.948475,71.886994,0.000000,38687.617160
7,27,0.000684,0.1365,0.0003,0,220250.627318,140.348928,30049.421094,61.545015,0.000000,30251.315037
8,28,0.000722,0.1254,0.0003,0,189999.312281,128.559492,23813.740438,53.407050,0.000000,23995.706980
9,29,0.000763,0.1152,0.0003,0,166003.605301,119.347552,19113.452588,46.914999,0.000000,19279.715139
